# SAURIA SIMULATION EXAMPLE

## Introduction

In this notebook, we'll take a look at a basic example for simulating convolutions with SAURIA via RTL emulation with Verilator.

To perform simulations with SAURIA, we use a custom made framework based on Python and Verilator. Experiments and tests are written and executed in Python, and our Verilator testbench emulates the execution of convolutions in hardware. As depicted in the figure below, we use temporary .txt files in order to transfer data between the two environments.

<img src="figures/methodology.svg" alt="drawing" width="750"/>

As an example, we will use this framework to execute an arbitrary convolution operation with SAURIA and obtain the results computed by the hardware and its performance statistics.

&#x26A0; *IMPORTANT* &#x26A0;

Before running the cells below, please make sure that all the requirements and installation steps described in the README have been performed, and that the Python kernel selected in the Jupyter Notebook is "sauria-env" from 'Python/sauria-env/bin/python'.

In [ ]:
# Let's import the dependencies we need
import numpy as np
import sys
import torch
import dotenv

# LOAD SYSTEM ENVIRONMENT VARIABLES - To compile Verilator from here
dotenv.load_dotenv('../env', override=True)

sys.path.insert(1, './../') # To find the libraries inside Python folder
import src.hw_versions as hwv
import src.sauria_lib as slib

## Build SAURIA in Verilator

Before starting the test, we need to make sure that the Verilator model of our hardware has been compiled, so that we can perform RTL emulations.

You can do that by executing the following commands in your terminal:

```
cd test/verilator
./compile_sauria.sh
```

*Alternatively*, you can also run the following code cell, which does the same from the context of this Notebook, via `subprocess`.

This only needs to be performed once for every version of the RTL, so if you have built SAURIA before, you can skip this step!

In [ ]:
import os
import subprocess

# Version - See 'Python/versions/hw_versions.py'
sauria_version = 'FP16_8x16'

cwd = os.getcwd()

os.chdir("../../test/verilator")
f1 = open("verilator_compile.log","w")
subprocess.call(["sh","./compile_sauria.sh",sauria_version],stdout=f1)
os.chdir(cwd)

## Define an example convolution

To test SAURIA with some random data, let's define an arbitrary convolution. We will choose small channel and spatial sizes so that the emulation does not take very long.

As a reference, we will perform the convolution using Pytorch, and later compare these results to the output of SAURIA.

In [ ]:
# Convolution options:
C_in = 32       # Input Channels
C_out = 32      # Output Channels
Kh,Kw = 3,3     # Kernel size
s = 1           # Strides
d = 1           # Dilation coefficient
#p = 0           # Padding (UNSUPPORTED ATM!)

# Define pytorch convolutional layer (randomly initialized weights & biases)
B_conv_torch = torch.nn.Conv2d(C_in, C_out, (Kh, Kw), stride=s, dilation=d)

# Output tensor shape
Cw = 8         # Output tensor width
Ch = 8          # Output tensor height

# Input tensor shape determined by output tensor shape
Aw = (1+s*(Cw-1)) + (1+d*(Kw-1)) - 1
Ah = (1+s*(Ch-1)) + (1+d*(Kh-1)) - 1

# Randomly generate input tensor
tensor_A_torch = torch.randn(C_in, Ah, Aw, dtype=torch.float32)

# Perform convolution with Pytorch and print result
tensor_C_torch = B_conv_torch(tensor_A_torch)

print(tensor_C_torch.shape)
print(tensor_C_torch[:3,:3,:3])

## Convert tensors to numpy

The SAURIA Python library works with numpy arrays, so first we need to convert the Pytorch tensors into numpy.

In [ ]:
# Input tensor is the same, but converted to numpy
tensor_A = np.array(tensor_A_torch.detach())

# Weights tensor is obtained from the conv layer (randomly generated)
tensor_B = np.array(B_conv_torch.weight.detach())

# Bias can be added by preloading data into the array
# (This is OPTIONAL! It adds the cost of replicating the data!)
# (However, it is useful as an example of data preloading)
bias_numpy = np.array(B_conv_torch.bias.detach())
preload_C = np.zeros([C_out,Ch,Cw])
preload_C[:,:,:] = np.reshape(bias_numpy,[C_out,1,1])

# Convert result into numpy to compare
tensor_C = np.array(tensor_C_torch.detach())

print(tensor_C.shape)
print(tensor_C[:3,:3,:3])

## Define the SAURIA configuration

Before running the hardware emulation, we need to generate the configuration dictionaries that describe the operation of SAURIA. Our library defines 3 configuration structures:

* **HW_PARAMS** - Describes the HW parameters of a specific SAURIA configuration. Changing these values requires changing the hardware, and therefore recompiling the Verilator model. Currently, only the "FP16_8x16" is fully integrated into our framework (*more coming soon!*).

* **TILING_DICT** - Describes the size and configuration of the tiles that will be used to partition the full convolution so that it fits into the hardware. Ideally, this should be done by a tiling algorithm that automatically discovers the optimal tile size to maximize throughput, but that implementation is still a work-in-progress, so for now we will define it by hand.

* **CONV_DICT** - Describes all the convolution-related variables that SAURIA needs to perform an operation. It can be obtained by using the function `get_conv_dict`:

In [ ]:
# Dictionary of hardware parameters describing the version of SAURIA
HW_PARAMS = hwv.get_params(sauria_version)

# Array with the tensor shapes to compute
tensor_shapes = [tensor_A.shape, tensor_B.shape, tensor_C.shape]

# Dictionary describing the tiling sizes
TILING_DICT = {
    'C_tile_shape'  :   [32,4,8],  #[C_out, Ch, Cw]
    'tile_cin'      :   32,
    'X_used'        :   16,
    'Y_used'        :   8
}

# Dictionary fully describing the convolution to compute
CONV_DICT = slib.get_conv_dict(tensor_shapes, TILING_DICT, HW_PARAMS, d=d, s=s, preloads=True)

print(CONV_DICT)

## RTL Emulation & Performance metrics

Now that everything is defined, we can use our library to run the convolution using SAURIA via Verilator emulation.

With the option `print_statistics` set to `True`, the function will print some statistics of the execution, such as the average throughput, total numer of cycles and number of stalls. Additionally, setting `generate_vcd=True` will generate a VCD file with the waveforms of the accelerator, which can be used later for analysis.


*(NOTE: generating the VCD file makes emulation much slower, and may result in gigantic files, so be careful with long simulations!)*

In [ ]:
SAURIA_outputs, SAURIA_stats = slib.Conv2d_SAURIA(tensor_A, tensor_B, preload_C, tensor_C, CONV_DICT, HW_PARAMS, generate_vcd=True, print_statistics=True, silent=False)

&#x1F914; Note that the performance of SAURIA in this configuration is not stellar: its average utilization is about 55%. As indicated by the memory stall cycles, this happens because, in this particular configuration, SAURIA spends over 40% of the time waiting for configuration or memory transactions to finish (mostly memory).

In this case, the main reason is that the convolution and tile shapes are too small to fully exploit the capabilities of SAURIA. Note that the memory utilization of the internal memories of SAURIA is quite small, so we are not achieving the full potential of the accelerator. The last code cell of this Notebook contains an example with a larger convolution, which achieves a much higher utilization and throughput. &#x1F680;

## Compare results

Now that we have some results from SAURIA, we can compare them to the ones we obtained directly using Pytorch.

The results will not be the same for two main reasons:

1. Pytorch uses FP32 values, while the base configuration of SAURIA used in this example uses FP16. The difference in precision will cause SAURIA's outputs to be less accurate.

2. In floating-point arithmetic, the order of operations matters. Hence, even if the precisions were the same, we would get slightly different results.

In [ ]:
# Print and compare to Pytorch result
print("From Pytorch:")
print(tensor_C[:3,:3,:3])

print("\nFrom SAURIA:")
print(SAURIA_outputs[:3,:3,:3])

print("\nAverage absolute error:")
print(np.abs(SAURIA_outputs - tensor_C).mean())

## Check the waveforms

We can use the open-source software `gtkwave` to explore the waveforms exported as a VCD file, by running the following commands on our terminal:

```
cd test/verilator
gtkwave -g new.vcd
```

With the waves we can confirm that the reason for the low utilization is the size of the convolution:

<img src="figures/gtwaves.png" alt="drawing" width="900"/>

The initial ramp-up time of bringing the frist tiles into the accelerator takes about 40% of the total time. Since the computation is short and the number of tiles is low, this ramp-up time is not amortized enough!

## Test with larger convolution

As promised, now let's repeat the experiment with a slightly larger convolution and tile size. The function `generate_and_run_test` will automatically generate random stimuli so that we don't need to do that again.

The emulation may take a while, so go grab a coffee in the meantime! &#x2615;

In [ ]:
# Convolution options:
C_in = 64       # Input Channels
C_out = 128     # Output Channels
Kh,Kw = 3,3     # Kernel size
s = 1           # Strides
d = 1           # Dilation coefficient
p = 0           # Padding (UNSUPPORTED ATM!)

# Output tensor shape
Cw = 32         # Output tensor width
Ch = 32         # Output tensor height

# Input tensor shape determined by output tensor shape
Aw = (1+s*(Cw-1)) + (1+d*(Kw-1)) - 1
Ah = (1+s*(Ch-1)) + (1+d*(Kh-1)) - 1

tensor_shapes = [
    [C_in,Ah,Aw],           # A tensor (inputs)
    [C_out,C_in,Kh,Kw],     # B tensor (outputs)
    [C_out,Ch,Cw]           # C tensor (psums)
]

TILING_DICT = {
    'C_tile_shape'  :   [32,8,32],  #[C_out, Ch, Cw]
    'tile_cin'      :   32,
    'loop_order'    :   1,
    'X_used'        :   16,
    'Y_used'        :   8
}

SAURIA_outputs_large, SAURIA_stats_large, partial_macs = slib.generate_and_run_test(tensor_shapes, TILING_DICT, d, s, HW_PARAMS, preload=True, generate_vcd=False, print_statistics=True, silent=False)